## Jupyter (Python) notebook to (1) screen USGS basins based on a variety of selection criteria and then (2) summarize their characteristics in an output .csv table


In [1]:
# Import required libraries for the analyses

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import georasters as gr
from scipy import stats
import warnings
warnings.filterwarnings("ignore")


In [2]:
# We rely on the National Hydrography Dataset for USGS basin boundaries
# Original file obtained from: 
# https://water.usgs.gov/GIS/metadata/usgswrd/XML/streamgagebasins.xml
# and reprojected into UTM 

gages = gpd.read_file('../data/basins/basins18_utm.shp')

In [3]:
# polygon of the non-usgs (Eel River CZO) Dry Creek watershed
dry_ck = gpd.read_file('../data/dry_creek_polygon/dry.shp')

In [4]:
#get the stream gage names
gage_names = gpd.read_file('../data/USGS_gages/USGS_Streamgages-NHD_Locations.shp')


In [5]:
#Create a dictionary that maps the USGS gage ID number to the station name
id_name = dict(zip(gage_names.SITE_NO,gage_names.STATION_NM))

In [6]:
#Read in information about dams from the GAGES II database, find sites with 0 dams
dam_data = pd.read_csv('../data/basinchar_and_report_sept_2011/spreadsheets-in-csv-format/conterm_hydromod_dams.txt')
no_dams = dam_data[dam_data['NDAMS_2009']==0]


In [7]:
#Read in information about climate from the GAGES II database, find sites with < 20 percent precip as snow

climate_data = pd.read_csv('../data/basinchar_and_report_sept_2011/spreadsheets-in-csv-format/conterm_climate.txt')
rain_dominated = climate_data[climate_data['SNOW_PCT_PRECIP']<20]


In [8]:
#find sites with < 10% of precip falling between may and september

climate_data['summer_precip_pct'] = 100*(climate_data.loc[:,'MAY_PPT7100_CM':'SEP_PPT7100_CM'].sum(axis=1)/climate_data.loc[:,'JAN_PPT7100_CM':'DEC_PPT7100_CM'].sum(axis=1))
winter_wet_summer_dry = climate_data[climate_data['summer_precip_pct']<10]


In [9]:
# find sites with good flow records during months of interest
flow_rec_data = pd.read_csv('../data/basinchar_and_report_sept_2011/spreadsheets-in-csv-format/conterm_flowrec.txt')
flow_rec_data['goodyears'] = flow_rec_data.iloc[:,-8:-1].sum(axis=1)
good_flow_rec = flow_rec_data[flow_rec_data['goodyears']== 7]


In [10]:
#narrow the list of study gages (represented by their station IDS, or STAID)
#by intersecting the set of each criteria. print the number of stations within the subset each time

study_gages = set(no_dams.STAID).intersection(rain_dominated.STAID)
print(len(study_gages))
study_gages = study_gages.intersection(good_flow_rec.STAID)
print(len(study_gages))
study_gages = study_gages.intersection(winter_wet_summer_dry.STAID)
print(len(study_gages))
study_gages = study_gages.intersection(gages.SITE_NO.astype(int))
print(len(study_gages))


1240
592
71
64


In [11]:
study_gages.add(11475800) # add back in the gage 11475800 SF EEL R A LEGGETT, which was excluded due to presence of dam
#this dam is only operational in the summer, however, and does not affect the winter storage calculations

In [12]:
# Read in the National Land Cover Database (NLCD)
#source: https://www.mrlc.gov/nlcd2011.php
#original raster file has been converted to UTM and clipped to California
landcover = gr.from_file('../data/landcover/cal_landcover_utm10N.tif')


In [13]:
#determine the percentage of 'developed' landcover in each polygon
#create empty list
land_cover_developed_pct = []
missing_gages=[]
#loop through study gages, 
gages['SITE_NO'] = gages['SITE_NO'].astype(str)
for gage in list(study_gages):
    gage = str(gage)
    #get the basin polygon
    basin_polygon = gages[gages.SITE_NO==gage].geometry
    #clip the landcover raster to this polygon
    r_basin = landcover.clip(basin_polygon)[0]
    #convert all the pixel values of this raster to one long array
    pixel_values =  r_basin.raster.data.flatten().astype(int)
    #determine the percent developed based on the NLCD legend, https://www.mrlc.gov/nlcd11_leg.php
    pct_developed = 100*(np.count_nonzero(pixel_values == 22)+
                         np.count_nonzero(pixel_values == 23)+
                         np.count_nonzero(pixel_values == 24))/np.count_nonzero(pixel_values)
    land_cover_developed_pct.append(pct_developed)
  
            
land_cover_developed_pct=np.array(land_cover_developed_pct)

In [14]:
#subset the study gages again to exclude sites with > 10% developed cover
study_gages = np.array(list(study_gages))[land_cover_developed_pct<10]

In [15]:

#Perform a similar operation, this time with 'cultivated' lands

land_cover_cultivated_pct = []
for gage in list(study_gages):
    gage = str(gage)
    pct_cultivated = 0
    basin_polygon = gages[gages.SITE_NO==gage].geometry
    r_basin = landcover.clip(basin_polygon)[0]
    pixel_values =  r_basin.raster.data.flatten().astype(int)

    pct_cultivated = 100*(float(np.count_nonzero(pixel_values == 82))/np.count_nonzero(pixel_values))
    land_cover_cultivated_pct.append(pct_cultivated)
land_cover_cultivated_pct=np.array(land_cover_cultivated_pct)

In [16]:
study_gages = np.array(list(study_gages))[land_cover_cultivated_pct<5]

In [17]:
#The site_fires.csv data table lists the intersection of every California USGS gaged basin with a shapefile of burned area polygons
# The resulting table lists the fraction of the basin that was burned in a particular year
# The accompanying notebook 'get_site_fires.ipynb' shows how the .csv was created from the original data
#original data source: http://frap.fire.ca.gov/data/frapgisdata-sw-fireperimeters_download

fires = pd.read_csv('../data/site_fires.csv')
fires.head()


fire_threshold_year = 1990
fire_threshold_size_fraction = 0.2

#Only consider fires post 1990 - 
recent_fires = fires[fires.fire_year >=fire_threshold_year]

site_group = recent_fires.groupby(['USGS Basin'])

#sum the total fraction of each burned catchment (this may include 'double counting' of areas that burned twice)
total_fraction_burned = site_group['fraction_catchment'].agg('sum').reset_index()

#subset the places where more than 20% of the basin burned
recent_large_fires = total_fraction_burned[total_fraction_burned.fraction_catchment>fire_threshold_size_fraction]
#get the resulting list of sites and filter them from the study basins
burned_sites = np.unique(recent_large_fires['USGS Basin'])
study_gages = set(study_gages)-set(burned_sites)


In [18]:
# Perform a similar analysis where the basins have been intersected with a geospatial logging layer
#this lists locations where a timber harvest plan of 'clearcut' or 'commercial thin completed' has been achieved

#The original data source is: ftp://ftp.fire.ca.gov/forest/Statewide_Timber_Harvest/
# these layers were reduced with the 'tabulate intersection' tool in ArcGIS, http://pro.arcgis.com/en/pro-app/tool-reference/analysis/tabulate-intersection.htm

logging = pd.read_csv('../data/Cal_USGS_Basins_TabulatedIntersection_THP_Clearcut_or_CommercialThin_Completed.csv')

logging_threshold_size_fraction = 20 #%


site_group = logging.groupby(['SITE_NO'])

total_fraction_logged = site_group['PERCENTAGE'].agg('sum').reset_index()

recent_large_fires = total_fraction_logged[total_fraction_logged.PERCENTAGE>logging_threshold_size_fraction]

logged_sites = np.unique(recent_large_fires['SITE_NO'])
study_gages = set(study_gages)-set(logged_sites)


In [19]:
#add the non-USGS Dry Ck. (Eel River Critical Zone Observatory) to the list
study_gages.add('0000') #add dry ck
gages.loc[len(gages)] = [np.nan,'0000',1.37,dry_ck.iloc[0].geometry]
id_name['0000'] = 'Dry Creek (ERCZO)'

In [20]:
#Remove two gages that have excessive water extraction, based on the USGS Water Year Summary Remarks
study_gages.remove(11169500) #Saratoga Ck - water diverted for municipal use by San Jose Water Works
study_gages.remove(11160000) #Soquel Ck - many diversions upstream from station for irrigatino

In [21]:
len(study_gages)

26

# The gauge selection process is complete. The next step (Part II) is to create a summary table that lists climatic, physiographic, geologic, etc. information about each basin

In [22]:
#Create a new dataframe and populate the ID and name columns
df = pd.DataFrame(index = list(study_gages))
df['ids'] = df.index.astype(str)
df['Name'] = df['ids'].map(id_name)

In [23]:
#Read in rasters of Elevation, temperature & Precip (PRISM data), and % tree canopy cover (National land cover database)

In [24]:
filename = '../data/Cal90mDEM_UTM.tif'
elev = gr.from_file(filename)


In [25]:
filename = '../data/PRISM_800m_30yr_TMEAN_UTM.tif'
temp = gr.from_file(filename)


In [26]:
filename = '../data/CAL_canopy_utm.tif'
canopy = gr.from_file(filename)


In [27]:
filename = '../data/PRISM_800m_30yr_PPT_UTM.tif'
precip = gr.from_file(filename)


In [28]:
#Determine the average basin canopy %, precip, etc., and add to the dataframe

In [29]:
canopy_means = []
for gage in df['ids']:    
    basin_polygon = gages[gages.SITE_NO==gage].geometry
    r_basin = canopy.clip(basin_polygon)[0]
    canopy_means.append(r_basin.mean())
df['Basin Mean Canopy Cover (%)'] = canopy_means
df['Basin Mean Canopy Cover (%)'] = df['Basin Mean Canopy Cover (%)'].astype(int)

In [30]:
precip_means = []
for gage in df['ids']:    
    basin_polygon = gages[gages.SITE_NO==gage].geometry
    r_basin = precip.clip(basin_polygon)[0]
    precip_means.append(r_basin.mean())
df['Basin Mean Annual Precipitation (mm)'] = precip_means
df['Basin Mean Annual Precipitation (mm)'] = df['Basin Mean Annual Precipitation (mm)'].astype(int)

In [31]:
temp_means = []
for gage in df['ids']:    
    basin_polygon = gages[gages.SITE_NO==gage].geometry
    r_basin = temp.clip(basin_polygon)[0]
    temp_means.append(r_basin.mean())
df['Basin Mean Annual Temperature (deg. C)'] = temp_means
df['Basin Mean Annual Temperature (deg. C)'] = df['Basin Mean Annual Temperature (deg. C)'].round(1)

In [32]:
elev_means = []
for gage in df['ids']:    
    basin_polygon = gages[gages.SITE_NO==gage].geometry
    r_basin = elev.clip(basin_polygon)[0]
    elev_means.append(r_basin.mean())
df['Basin Mean Elevation (m)'] = elev_means
df['Basin Mean Elevation (m)'] = df['Basin Mean Elevation (m)'].astype(int)

In [33]:
# Create a dictionary that maps the land cover classification code to its description
landcover_legend = pd.read_csv('../data/USGS_LANDCOVER_LEGEND.csv')
landcover_dict = dict(zip(landcover_legend.Class,landcover_legend['Short Description']))

In [34]:
land_cover_modes = []
for gage in df['ids']:    
    basin_polygon = gages[gages.SITE_NO==gage].geometry
    r_basin = landcover.clip(basin_polygon)[0]
    land_cover_modes.append(stats.mode(r_basin.raster.data.flatten())[0][0])

df['Land Cover Class'] = land_cover_modes
df['Land Cover Class'] = df['Land Cover Class'].astype(int)
df['Dominant Land Cover'] = df['Land Cover Class'].map(landcover_dict)
df.drop('Land Cover Class',axis=1,inplace=True)


In [35]:
#add in basin area, lat, and lon
basin_areas = []
station_lats = []
station_lons = []
for gage in df['ids']:    
    basin_area_mi2 = gages[gages.SITE_NO==gage].SQMI
    basin_area_km2 = basin_area_mi2 * 2.58999
    basin_areas.append(basin_area_km2)
    if gage == '0000':
        station_lats.append(39.5754)
        station_lons.append(-123.4642)
    else:
        station_lats.append(gage_names[gage_names['SITE_NO']==gage].LAT_SITE.values[0])
        station_lons.append(gage_names[gage_names['SITE_NO']==gage].LON_SITE.values[0])

df['Basin area (km^2)'] = basin_areas
df['Basin area (km^2)'] = df['Basin area (km^2)'].astype(int)
df['Gage Latitude'] = station_lats
df['Gage Longitude'] = station_lons

In [36]:
# Add in the dominant geology, based on this source
# https://mrdata.usgs.gov/geology/state/state.php?state=CA
# Jennings, C.W., Strand, R.G., and Rogers, T.H., 1977, Geologic map of California: California Division of Mines and Geology, scale 1:750,000. 

# Open-File Report (2005-1305)
# Preliminary integrated geologic map databases for the United States
# Western States: California, Nevada, Arizona, Washington, Oregon, Idaho, and Utah
# Version 1.3
# Updated December 2007
# By: Steve Ludington, Barry C. Moring, Robert J. Miller, Paul A. Stone, Arthur A. Bookstrom, David R. Bedford, James G. Evans, Gordon A. Haxel, Contstance J. Nutt, Kathryn S. Flyn and Melanie J. Hopkins 
#The basin geology has been intersected previously in ArcGIS, creating the following output table
gages_geol = pd.read_csv('../data/StudyBasins_CalGeol_ArcGIS-Intersect.csv')
gages_geol.head()

,OBJECTID,FID_study_basins,SITE_NO,SQMI,ABS_DIFF,FID_cageol_poly_dd,AREA,PERIMETER,CAGEOL_DD_,CAGEOL_DD1,ORIG_LABEL,SGMC_LABEL,UNIT_LINK,SOURCE,UNIT_AGE,ROCKTYPE1,ROCKTYPE2,Shape_Length,Shape_Area
0,1,0,11046300,80.939803,0.00173,11254,0.344374,18.111147,11256,12009,grMz,grMZ2;0,CAgrMZ2;0,CA001,Middle Jurassic to Late Cretaceous,tonalite,quartz diorite,120867.218635,1.166864e+08
1,2,0,11046300,80.939803,0.00173,11338,0.005078,0.591551,11340,4290,K,K1;0,CAK1;0,CA001,Early to Late Cretaceous,mudstone,sandstone,6862.682224,6.168842e+05
2,3,0,11046300,80.939803,0.00173,11348,0.000055,0.038878,11350,4289,Qv,Qv8;0,CAQv8;0,CA001,Quaternary,basalt,,1768.828032,1.421323e+05
3,4,0,11046300,80.939803,0.00173,11349,0.000724,0.158108,11351,4295,gb,gb2;0,CAgb2;0,CA001,Triassic to Cretaceous,gabbro,diorite,16370.143020,7.379422e+06
4,5,0,11046300,80.939803,0.00173,11350,0.002738,0.450280,11352,4300,J,J4;0,CAJ4;0,CA001,Paleozoic(?) to Late Jurassic,argillite,graywacke,26231.713077,1.386198e+07


In [37]:
# read in a metadata file that pairs the geology unit code with the unit description
geol_unit_descrip = pd.read_csv('../data/CAunits.csv')
geol_unit_descrip.head()

,STATE,ORIG_LABEL,MAP_SYM1,MAP_SYM2,UNIT_LINK,PROV_NO,PROVINCE,UNIT_NAME,UNIT_AGE,UNITDESC,STRAT_UNIT,UNIT_COM,MAP_REF,ROCKTYPE1,ROCKTYPE2,ROCKTYPE3,UNIT_REF
0,CA,C,C1,C1;0,CAC1;0,0,NaN,"Carboniferous marine rocks, unit 1 (Western Mo...",Late Proterozoic to Pennsylvanian,"Shale, sandstone, conglomerate, limestone, dol...","Furnace Limestone, Oro Grande Series, Bird Spr...",Southern California (San Bernardino Mountains ...,CA001,marble,limestone,dolostone (dolomite); schist; quartzite; hornfels,CA001 CA002 CA016 CA032 CA046 CA067 CA098 CA09...
1,CA,C,C2,C2;0,CAC2;0,0,NaN,"Carboniferous marine rocks, unit 2 (SE Califor...",Mississippian to Early Permian,"Shale, sandstone, conglomerate, limestone, dol...","Anvil Spring Fm., Bird Spring Fm. (part), Tihv...",Southeastern California carbonate assemblage (...,CA001,limestone,mudstone,sandstone,CA001 CA002 CA006 CA008 CA012 CA025 CA043 CA07...
2,CA,C,C3,C3;0,CAC3;0,0,NaN,"Carboniferous marine rocks, unit 3 (SE Califor...",Late Devonian to Early Permian,"Shale, sandstone, conglomerate, limestone, dol...","Keeler Canyon Fm., Tihvipah Limestone, Rest Sp...",Southeastern California clastic assemblage (no...,CA001,shale,limestone,argillite; siltstone; conglomerate; chert,CA001 CA002 CA006 CA007 CA011 CA043 CA072 CA07...
3,CA,C,C4,C4;0,CAC4;0,0,NaN,"Carboniferous marine rocks, unit 4 (Eastern Kl...",Mississippian to Early Permian,"Shale, sandstone, conglomerate, limestone, dol...","Baird Fm., Bass Mountain Diabase (part), Bragd...",Eastern Klamath Mountains. Consists primarily ...,CA001,mudstone,pyroclastic,conglomerate; sandstone; limestone; greenstone,CA001 CA002 CA013 CA028 CA034 CA036 CA039 CA15...
4,CA,C,C5,C5;0,CAC5;0,0,NaN,"Carboniferous marine rocks, unit 5 (Northweste...",Paleozoic or Mesozoic,"Shale, sandstone, conglomerate, limestone, dol...",NaN,Northwestern Sierra Nevada. Composed of quartz...,CA001,quartzite,chert,NaN,CA001 CA002 CA030 CA179 CA180


In [38]:
#Get the largest by area..
dominant_geol = gages_geol.sort_values('Shape_Area', ascending=False).drop_duplicates(['SITE_NO'])
#merge with the unit description
merged_geol = pd.merge(dominant_geol, geol_unit_descrip,how='inner',on='UNIT_LINK')

In [39]:
merged_geol['SITE_NO']=merged_geol['SITE_NO'].astype(str)

In [40]:
#add dry creek's special #
merged_geol.at[19,'SITE_NO'] = '0000'


In [41]:
#Merge the geology dataframe with the other data
merged_df = pd.merge(df,merged_geol.loc[:,['SITE_NO','UNITDESC']],left_on='ids',right_on='SITE_NO')
merged_df.drop('ids',axis=1,inplace=True)
merged_df.rename(columns={'UNITDESC': 'Dominant lithology'}, inplace=True)

In [42]:
#Finally, output the summary data table
merged_df.to_csv('../data/Table S2.csv',index=False)